In [ ]:
import Ab as ab
import datetime as dt
import matplotlib.pyplot as plt


# Get data from yahoo finance
sd = dt.datetime(2001,1,1)
ed = dt.datetime(2023,3,2)
# 1x leveraged nasdaq 100 etf
qqq = ab.StockData('QQQ')
qqq.get_data_from_yfinance('QQQ', sd, ed)
#nasdaq stocks above 200 day moving average percentage

qld = ab.StockData('QLD')
qld.get_data_from_yfinance('QLD', sd, ed)

naa200r = ab.StockData('NAA200R')
naa200r.get_data_from_csv('data/NAA200R.csv')


In [ ]:
def sliding_test(ticker: ab.StockData, stg:ab.Strategy, window: int, sd: dt.datetime, ed: dt.datetime, run_strategy: bool = True):
    delta = ed - sd
    result = []
    for i in range(0, delta.days + 1, 30):
        sd_5y = sd + dt.timedelta(days=i)
        ed_5y = sd + dt.timedelta(days=i) + dt.timedelta(days=window)
        
        if run_strategy:
            stg.run_strategy(ticker, sd_5y, ed_5y)
        bt = None
        bt = ab.BackTest(sd_5y, ed_5y,1)
        bt.run_backtest(stg, ticker)
        
        result.append([sd_5y,bt.balance['Total'].iloc[-1], stg.name])
        print(result[-1])
        
    return result



In [ ]:
test_sd = dt.datetime(2001,1,1)
test_ed = dt.datetime(2018,3,1)
window = 365*5
bts = []

In [ ]:
buy_and_hold = ab.BuyAndHold()
bh_result = sliding_test(qqq,buy_and_hold,window, test_sd,test_ed,True)
bts.append(bh_result)

In [ ]:
naa200R_stg = ab.Threshold(signal_data = naa200r, indicator = 'SignalPrice', buy_threshold = 15, sell_threshold = 30, signal_ma_window = 40)
naa200r_rst_15_30_ma40 = sliding_test(qqq,naa200R_stg,window, test_sd,test_ed, True)
bts.append(naa200r_rst_15_30_ma40)

In [ ]:
threshold_strategy = ab.Threshold(signal_data = naa200r, indicator = 'SignalPrice', buy_threshold = 15, sell_threshold = 30)
naa200r_result = sliding_test(qqq,threshold_strategy,window, test_sd,test_ed, True)
bts.append(naa200r_result)

In [ ]:
ma_threshold = ab.MAThreshold(ma_window = 200, buy_threshold = 1.05, sell_threshold = 0.95)
ma_threshold_result = sliding_test(qqq,ma_threshold,window, test_sd,test_ed, True)
bts.append(ma_threshold_result)


In [ ]:
ma_threshold2 = ab.MAThreshold(ma_window = 200, buy_threshold = 1.03, sell_threshold = 0.95)
ma_threshold_result2 = sliding_test(qqq,ma_threshold2,window, test_sd,test_ed, True)
bts.append(ma_threshold_result2)


In [ ]:
macross_strategy = ab.MACross(short_window=50, long_window=200)
ma_threshold_result2 = sliding_test(qqq,macross_strategy,window, test_sd,test_ed, True)
bts.append(ma_threshold_result2)

In [ ]:

WeeklyMAThreshold = ab.WeeklyMAThreshold(ma_window =40,buy_threshold=1,sell_threshold=1)
WeeklyMAThreshold_result = sliding_test(qqq,WeeklyMAThreshold,window, test_sd,test_ed, True)
bts.append(WeeklyMAThreshold_result)

In [ ]:
for bt in bts:
    plt.plot([x[0] for x in bt], [x[1] for x in bt], lable=bt.name)
plt.legend()

In [ ]:
import numpy as np
print('bh avg:', np.mean([x[1] for x in bh_result]))
print('naa200r avg:', np.mean([x[1] for x in naa200r_result]))

print('ma_threshold avg:', np.mean([x[1] for x in ma_threshold_result]))
print('ma_threshold2 avg:', np.mean([x[1] for x in ma_threshold_result2]))
print('WeeklyMAThreshold avg:', np.mean([x[1] for x in WeeklyMAThreshold_result]))

print('bh std:', np.std([x[1] for x in bh_result]))
print('naa200r std:', np.std([x[1] for x in naa200r_result]))

print('ma_threshold avg:', np.std([x[1] for x in ma_threshold_result]))
print('ma_threshold2 avg:', np.std([x[1] for x in ma_threshold_result2]))
print('WeeklyMAThreshold avg:', np.std([x[1] for x in WeeklyMAThreshold_result]))


